In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_14976/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [7]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_18_9_6,0.997122,0.810120,0.988839,0.973237,0.989085,0.019244,1.269731,0.019995,0.039988,0.029991,0.543964,0.138723,1.002093,0.144628,121.901116,191.377038,"Hidden Size=[14], regularizer=0.01, learning_r..."
1,model_18_9_5,0.997118,0.809948,0.990241,0.975807,0.990241,0.019271,1.270881,0.017482,0.036148,0.026815,0.527238,0.138819,1.002096,0.144729,121.898325,191.374247,"Hidden Size=[14], regularizer=0.01, learning_r..."
2,model_18_9_7,0.997103,0.810259,0.987353,0.970533,0.987865,0.019375,1.268802,0.022657,0.044028,0.033342,0.560244,0.139193,1.002107,0.145119,121.887573,191.363495,"Hidden Size=[14], regularizer=0.01, learning_r..."
3,model_18_9_4,0.997087,0.809739,0.991542,0.978212,0.991319,0.019479,1.272277,0.015152,0.032555,0.023853,0.510077,0.139568,1.002119,0.145510,121.876807,191.352729,"Hidden Size=[14], regularizer=0.01, learning_r..."
4,model_18_9_8,0.997063,0.810369,0.985800,0.967725,0.986596,0.019641,1.268066,0.025437,0.048225,0.036831,0.576045,0.140148,1.002136,0.146114,121.860235,191.336157,"Hidden Size=[14], regularizer=0.01, learning_r..."
5,model_18_9_3,0.997025,0.809489,0.992719,0.980419,0.992302,0.019896,1.273951,0.013044,0.029258,0.021151,0.492523,0.141052,1.002164,0.147057,121.834493,191.310415,"Hidden Size=[14], regularizer=0.01, learning_r..."
8,model_18_9_2,0.996927,0.809192,0.993750,0.982387,0.993174,0.020550,1.275936,0.011197,0.026317,0.018757,0.474609,0.143351,1.002235,0.149454,121.769836,191.245758,"Hidden Size=[14], regularizer=0.01, learning_r..."
10,model_18_9_1,0.996789,0.808843,0.994609,0.984077,0.993914,0.021472,1.278268,0.009657,0.023791,0.016724,0.456394,0.146534,1.002335,0.152773,121.681983,191.157905,"Hidden Size=[14], regularizer=0.01, learning_r..."
16,model_18_9_0,0.996605,0.808436,0.995270,0.985444,0.994500,0.022700,1.280986,0.008474,0.021749,0.015111,0.437933,0.150664,1.002469,0.157078,121.570817,191.046739,"Hidden Size=[14], regularizer=0.01, learning_r..."
21,model_18_8_5,0.996500,0.806608,0.971322,0.997871,0.996149,0.023401,1.293213,0.029104,0.030518,0.029811,0.414870,0.152975,1.002545,0.159488,121.509921,190.985843,"Hidden Size=[14], regularizer=0.01, learning_r..."
